# Use of swig and numpy to compute the evolution of internal energy in a vNR scheme

In [1]:
import os

#### We will use numpy extensively

In [2]:
import numpy as np

#### Import the module created by swig as a classical python module

In [3]:
import vnrinternalenergy as vnr_ext


#### Import classical numpy modules for performance comparison purpose

In [4]:
from xvof.solver.functionstosolve.vnrenergyevolutionforveformulation import VnrEnergyEvolutionForVolumeEnergyFormulation
from xvof.solver.newtonraphson import NewtonRaphson
from xvof.equationsofstate.miegruneisen import MieGruneisen

## Creation of the data

In [5]:
pb_size = 100000

In [6]:
old_density = np.ndarray(pb_size, dtype=np.float64, order='c')

In [7]:
new_density = np.ndarray(pb_size, dtype=np.float64, order='c')

In [8]:
pressure= np.ndarray(pb_size, dtype=np.float64, order='c')

In [9]:
internal_energy = np.ndarray(pb_size, dtype=np.float64, order='c')

In [10]:
new_internal_energy = np.ndarray(pb_size, dtype=np.float64, order='c')

In [11]:
new_pressure = np.ndarray(pb_size, dtype=np.float64, order='c')

In [12]:
new_soundspeed = np.ndarray(pb_size, dtype=np.float64, order='c')

## Simple test of the module

#### Initialization of the data

In [13]:
old_density[:] = 7500.

In [14]:
new_density[:] = 9500.

In [15]:
pressure[:] = 1e+09

In [16]:
internal_energy[:] = 1e+06

In [17]:
new_internal_energy[:] = 0.

In [18]:
new_pressure[:] = 0.

In [19]:
new_soundspeed[:] = 0.

In [20]:
print new_pressure[495:505], new_internal_energy[495:505], new_soundspeed[495:505]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


#### Use the module created by swig as a classical python module

In [21]:
vnr_ext.launch_vnr_resolution(old_density, new_density, pressure, internal_energy, new_internal_energy, new_pressure, new_soundspeed)

In [22]:
print new_pressure[495:505], new_internal_energy[495:505], new_soundspeed[495:505]

[  5.10950604e+10   5.10950604e+10   5.10950604e+10   5.10950604e+10
   5.10950604e+10   5.10950604e+10   5.10950604e+10   5.10950604e+10
   5.10950604e+10   5.10950604e+10] [ 1731158.74241368  1731158.74241368  1731158.74241368  1731158.74241368
  1731158.74241368  1731158.74241368  1731158.74241368  1731158.74241368
  1731158.74241368  1731158.74241368] [ 5771.42229347  5771.42229347  5771.42229347  5771.42229347  5771.42229347
  5771.42229347  5771.42229347  5771.42229347  5771.42229347  5771.42229347]


## Test of performance of swig made module

In [23]:
new_internal_energy[:] = 0.

In [24]:
new_pressure[:] = 0.

In [25]:
new_soundspeed[:] = 0.

In [26]:
print new_pressure[495:505], new_internal_energy[495:505], new_soundspeed[495:505]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [27]:
%timeit vnr_ext.launch_vnr_resolution(old_density, new_density, pressure, internal_energy, new_internal_energy, new_pressure, new_soundspeed)

100 loops, best of 3: 9.43 ms per loop


In [28]:
print new_pressure[495:505], new_internal_energy[495:505], new_soundspeed[495:505]

[  5.10950604e+10   5.10950604e+10   5.10950604e+10   5.10950604e+10
   5.10950604e+10   5.10950604e+10   5.10950604e+10   5.10950604e+10
   5.10950604e+10   5.10950604e+10] [ 1731158.74241368  1731158.74241368  1731158.74241368  1731158.74241368
  1731158.74241368  1731158.74241368  1731158.74241368  1731158.74241368
  1731158.74241368  1731158.74241368] [ 5771.42229347  5771.42229347  5771.42229347  5771.42229347  5771.42229347
  5771.42229347  5771.42229347  5771.42229347  5771.42229347  5771.42229347]


## Test of performance of classical numpy module

In [29]:
function_to_vanish = VnrEnergyEvolutionForVolumeEnergyFormulation()
solver = NewtonRaphson(function_to_vanish)

In [30]:
my_variables = {'EquationOfState': MieGruneisen(),
                'OldDensity': old_density,
                'NewDensity': new_density,
                'Pressure': pressure,
                'OldEnergy': internal_energy}

In [31]:
function_to_vanish.setVariables(my_variables)


In [32]:
%timeit reference_solution = solver.computeSolution(internal_energy)

10 loops, best of 3: 42.8 ms per loop


In [33]:
reference_solution = solver.computeSolution(internal_energy)
print reference_solution[495:505]

[ 1731158.74241368  1731158.74241368  1731158.74241368  1731158.74241368
  1731158.74241368  1731158.74241368  1731158.74241368  1731158.74241368
  1731158.74241368  1731158.74241368]


## Test of behavior in case of wrong inputs

In [34]:
wrong_sized_density = np.ndarray(999, dtype=np.float64, order='c')

In [35]:
wrong_sized_density[:] = 9500.

In [36]:
new_internal_energy[:] = 0.

In [37]:
new_pressure[:] = 0.

In [38]:
new_soundspeed[:] = 0.

In [39]:
#print new_pressure[495:505], new_internal_energy[495:505], new_soundspeed[495:505]

In [40]:
#vnr_ext.launch_vnr_resolution(old_density, wrong_sized_density, pressure, internal_energy, new_internal_energy, new_pressure, new_soundspeed)